In [1]:
import polars as pl
from quantbullet.utils.encrypt import decrypt_variable_from_file

In [49]:
polars_df = pl.from_pandas(df)

In [10]:
df['mba_dlq_status'].value_counts()

mba_dlq_status
C    34115
0    33313
3      169
9      110
F       66
6       33
Name: count, dtype: int64

In [22]:
df[ df['mba_dlq_status'] == '9' ]['loan_id']
df[ df['loan_id'] == '000304023886' ].to_clipboard()

In [26]:
df['factor_date']

0        2025-08-01
1        2025-07-01
2        2025-06-01
3        2025-05-01
4        2025-04-01
            ...    
67801    2024-03-01
67802    2024-02-01
67803    2024-01-01
67804    2023-12-01
67805    2023-11-01
Name: factor_date, Length: 67806, dtype: object

In [27]:
df['paid_to_date']

0        2016-01-01
1        2016-01-01
2        2016-01-01
3        2016-01-01
4        2016-01-01
            ...    
67801    2024-02-01
67802    2024-01-01
67803    2023-12-01
67804    2023-11-01
67805    2023-10-01
Name: paid_to_date, Length: 67806, dtype: object

In [63]:
df['current_actual_bal']

0             0.00
1             0.00
2             0.00
3             0.00
4             0.00
           ...    
67801    266562.20
67802    266989.12
67803    267414.62
67804    267838.70
67805    268261.38
Name: current_actual_bal, Length: 67806, dtype: float64

In [68]:

polars_df.with_columns(
  months_to_paid = (
    ( pl.col( "paid_to_date" ).dt.year() - pl.col( "factor_date" ).dt.year() ) * 12
    + ( pl.col( "paid_to_date" ).dt.month() - pl.col( "factor_date" ).dt.month() )
    - 1
  )
).with_columns(
  dlq_months = pl.col( "months_to_paid" ).clip( lower_bound = 0 )
).with_columns(
  dlq_status = (
    pl.when( ( pl.col( "mba_dlq_status" ) == "0" ) | ( pl.col( "current_actual_bal" ) <= 0.001 ) )
      .then( pl.lit( "PD" ) )
    .when( pl.col( "mba_dlq_status" ) == "C" )
      .then( pl.lit( "C" ) )
    .when( pl.col( "mba_dlq_status" ) == "3" )
      .then( pl.lit( "M30" ) )
    .when( pl.col( "mba_dlq_status" ) == "6" )
      .then( pl.lit( "M60" ) )
    .when( pl.col( "mba_dlq_status" ) == "F" )
      .then( pl.lit( "FCLS" ) )
    .when( pl.col( "mba_dlq_status" ) == "R" )
      .then( pl.lit( "REO" ) )
    .when( pl.col( "mba_dlq_status" ) == "9" )
      .then(
        pl.when( pl.col( "dlq_months" ) <= 3 ).then( pl.lit( "M90" ) )
        .when( pl.col( "dlq_months" ) == 4 ).then( pl.lit( "M120" ) )
        .when( pl.col( "dlq_months" ) == 5 ).then( pl.lit( "M150" ) )
        .when( pl.col( "dlq_months" ) == 6 ).then( pl.lit( "M180" ) )
        .when( pl.col( "dlq_months" ) == 7 ).then( pl.lit( "M210" ) )
        .when( pl.col( "dlq_months" ) == 8 ).then( pl.lit( "M240" ) )
        .otherwise( pl.lit( "M270P" ) )
      )
    .otherwise( pl.lit( "X" ) )
  )
)

deal_name,pool_id,factor_date,age,current_interest_rate,current_actual_bal,current_scheduled_bal,date_referred_to_foreclosure,paid_to_date,mba_dlq_status,loan_id,product_type,property_state,lien,postal_code,first_payment_date,maturity_date,original_loan_term,initial_rate,close_rate,orig_bal,close_bal,prepayment_penalty_indicator,fico,dti_ratio,pmi,servicer,origination_date,orig_ltv,number_of_units,loan_purpose,loan_source,property_type,occ_type,first_time_buyer,months_to_paid,dlq_months,dlq_status
str,str,date,i64,f64,f64,f64,date,date,str,str,str,str,i64,str,date,date,i64,f64,f64,f64,f64,str,i64,f64,f64,str,date,f64,i64,str,str,str,str,str,i32,i32,str
"""JPMMT 2014-1""","""1PU""",2025-08-01,146,4.0,0.0,0.0,null,2016-01-01,"""0""","""110313419 ""","""Fixed 1st""","""CA""",1,"""92653""",2013-07-01,2043-06-01,360,4.0,4.0,825000.0,815378.88,"""0""",732,30.0,0.0,"""RESURGENT CAPITAL """,2013-05-08,75.0,1,"""C""","""C""","""PU""","""O""","""Y""",-116,0,"""PD"""
"""JPMMT 2014-1""","""1PU""",2025-07-01,145,4.0,0.0,0.0,null,2016-01-01,"""0""","""110313419 ""","""Fixed 1st""","""CA""",1,"""92653""",2013-07-01,2043-06-01,360,4.0,4.0,825000.0,815378.88,"""0""",732,30.0,0.0,"""RESURGENT CAPITAL """,2013-05-08,75.0,1,"""C""","""C""","""PU""","""O""","""Y""",-115,0,"""PD"""
"""JPMMT 2014-1""","""1PU""",2025-06-01,144,4.0,0.0,0.0,null,2016-01-01,"""0""","""110313419 ""","""Fixed 1st""","""CA""",1,"""92653""",2013-07-01,2043-06-01,360,4.0,4.0,825000.0,815378.88,"""0""",732,30.0,0.0,"""RESURGENT CAPITAL """,2013-05-08,75.0,1,"""C""","""C""","""PU""","""O""","""Y""",-114,0,"""PD"""
"""JPMMT 2014-1""","""1PU""",2025-05-01,143,4.0,0.0,0.0,null,2016-01-01,"""0""","""110313419 ""","""Fixed 1st""","""CA""",1,"""92653""",2013-07-01,2043-06-01,360,4.0,4.0,825000.0,815378.88,"""0""",732,30.0,0.0,"""RESURGENT CAPITAL """,2013-05-08,75.0,1,"""C""","""C""","""PU""","""O""","""Y""",-113,0,"""PD"""
"""JPMMT 2014-1""","""1PU""",2025-04-01,142,4.0,0.0,0.0,null,2016-01-01,"""0""","""110313419 ""","""Fixed 1st""","""CA""",1,"""92653""",2013-07-01,2043-06-01,360,4.0,4.0,825000.0,815378.88,"""0""",732,30.0,0.0,"""RESURGENT CAPITAL """,2013-05-08,75.0,1,"""C""","""C""","""PU""","""O""","""Y""",-112,0,"""PD"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""JPMMT 2023-9""","""ETW""",2024-03-01,23,4.0,266562.2,266134.86,null,2024-02-01,"""C""","""000303511243""","""Fixed 1st""","""WI""",1,""" """,2022-05-01,2052-04-01,360,4.0,4.0,275625.0,268261.38,"""0""",767,42.4,0.0,"""SHELLPOINT """,2022-03-01,75.0,1,"""P""","""R""","""CO""","""S""","""N""",-2,0,"""C"""
"""JPMMT 2023-9""","""ETW""",2024-02-01,22,4.0,266989.12,266563.2,null,2024-01-01,"""C""","""000303511243""","""Fixed 1st""","""WI""",1,""" """,2022-05-01,2052-04-01,360,4.0,4.0,275625.0,268261.38,"""0""",767,42.4,0.0,"""SHELLPOINT """,2022-03-01,75.0,1,"""P""","""R""","""CO""","""S""","""N""",-2,0,"""C"""
"""JPMMT 2023-9""","""ETW""",2024-01-01,21,4.0,267414.62,266990.12,null,2023-12-01,"""C""","""000303511243""","""Fixed 1st""","""WI""",1,""" """,2022-05-01,2052-04-01,360,4.0,4.0,275625.0,268261.38,"""0""",767,42.4,0.0,"""SHELLPOINT """,2022-03-01,75.0,1,"""P""","""R""","""CO""","""S""","""N""",-2,0,"""C"""


In [73]:
df['paid_to_date']

0        2016-01-01
1        2016-01-01
2        2016-01-01
3        2016-01-01
4        2016-01-01
            ...    
67801    2024-02-01
67802    2024-01-01
67803    2023-12-01
67804    2023-11-01
67805    2023-10-01
Name: paid_to_date, Length: 67806, dtype: object

In [74]:
import pandas as pd
import numpy as np

# assume df is your pandas DataFrame

df['factor_date'] = pd.to_datetime(df['factor_date'])
df['paid_to_date'] = pd.to_datetime(df['paid_to_date'])

# months_to_paid
df["months_to_paid"] = (
    (df["paid_to_date"].dt.year - df["factor_date"].dt.year) * 12
    + (df["paid_to_date"].dt.month - df["factor_date"].dt.month)
    - 1
)

# clip negative to zero
df["dlq_months"] = df["months_to_paid"].clip(lower=0)

# now build dlq_status
conditions = [
    (df["mba_dlq_status"] == "0") | (df["current_actual_bal"] <= 0.001),
    (df["mba_dlq_status"] == "C"),
    (df["mba_dlq_status"] == "3"),
    (df["mba_dlq_status"] == "6"),
    (df["mba_dlq_status"] == "F"),
    (df["mba_dlq_status"] == "R"),
]

choices = ["PD", "C", "M30", "M60", "FCLS", "REO"]

df["dlq_status"] = np.select(conditions, choices, default="X")

# handle the mba == '9' special case with nested logic
mask9 = df["mba_dlq_status"] == "9"
df.loc[mask9 & (df["dlq_months"] <= 3), "dlq_status"] = "M90"
df.loc[mask9 & (df["dlq_months"] == 4), "dlq_status"] = "M120"
df.loc[mask9 & (df["dlq_months"] == 5), "dlq_status"] = "M150"
df.loc[mask9 & (df["dlq_months"] == 6), "dlq_status"] = "M180"
df.loc[mask9 & (df["dlq_months"] == 7), "dlq_status"] = "M210"
df.loc[mask9 & (df["dlq_months"] == 8), "dlq_status"] = "M240"
df.loc[mask9 & (df["dlq_months"] >= 9), "dlq_status"] = "M270P"


In [75]:
df

,deal_name,pool_id,factor_date,age,current_interest_rate,current_actual_bal,current_scheduled_bal,date_referred_to_foreclosure,paid_to_date,mba_dlq_status,...,orig_ltv,number_of_units,loan_purpose,loan_source,property_type,occ_type,first_time_buyer,months_to_paid,dlq_months,dlq_status
0,JPMMT 2014-1,1PU,2025-08-01,146,4.0,0.00,0.00,None,2016-01-01,0,...,75.0,1,C,C,PU,O,Y,-116,0,PD
1,JPMMT 2014-1,1PU,2025-07-01,145,4.0,0.00,0.00,None,2016-01-01,0,...,75.0,1,C,C,PU,O,Y,-115,0,PD
2,JPMMT 2014-1,1PU,2025-06-01,144,4.0,0.00,0.00,None,2016-01-01,0,...,75.0,1,C,C,PU,O,Y,-114,0,PD
3,JPMMT 2014-1,1PU,2025-05-01,143,4.0,0.00,0.00,None,2016-01-01,0,...,75.0,1,C,C,PU,O,Y,-113,0,PD
4,JPMMT 2014-1,1PU,2025-04-01,142,4.0,0.00,0.00,None,2016-01-01,0,...,75.0,1,C,C,PU,O,Y,-112,0,PD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67801,JPMMT 2023-9,ETW,2024-03-01,23,4.0,266562.20,266134.86,None,2024-02-01,C,...,75.0,1,P,R,CO,S,N,-2,0,C
67802,JPMMT 2023-9,ETW,2024-02-01,22,4.0,266989.12,266563.20,None,2024-01-01,C,...,75.0,1,P,R,CO,S,N,-2,0,C
67803,JPMMT 2023-9,ETW,2024-01-01,21,4.0,267414.62,266990.12,None,2023-12-01,C,...,75.0,1,P,R,CO,S,N,-2,0,C
67804,JPMMT 2023-9,ETW,2023-12-01,20,4.0,267838.70,267415.62,None,2023-11-01,C,...,75.0,1,P,R,CO,S,N,-2,0,C
